In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import chardet
import joblib

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
#         path = os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import kagglehub

# Download latest version
dataset_dir = kagglehub.dataset_download("abdulmalik1518/mobiles-dataset-2025")
print("Path to dataset files:", dataset_dir)

# Find the CSV file within the diectory
for filename in os.listdir(dataset_dir):
  if filename.endswith(".csv"):
    csv_path = os.path.join(dataset_dir, filename)
    break #Stop after finding the first CSV file
print("CSV file found at:", csv_path)

with open(csv_path, 'rb') as file:
    Chardet = chardet.detect(file.read())
print(Chardet['encoding'])

In [ ]:
df = pd.read_csv(csv_path, encoding=Chardet['encoding'])
df.head()

In [ ]:
df.tail()

#**Data cleaning and preparations**

#1: Check missing data

In [ ]:
df.info()

In [ ]:
df.isnull().sum()


 No missing data

#2: Check and remove duplications

In [ ]:
duplication = df.duplicated(keep='first').sum()
print(f'Duplicated rows: {duplication}')
df = df.drop_duplicates(keep='first')

#3: Review each column in dataframe

In [ ]:
for col in df.columns:
  print(f'Unique values in {col}:\n', sorted(df[col].dropna().unique()), '\n')

In [ ]:
# Convert ['Company Name', 'Model Name', 'Processor'] columns to categorical type
categorical_cols = ['Company Name', 'Model Name', 'Processor']
df[categorical_cols] = df[categorical_cols].astype('category')

# Convert 'Launched Year' to datetime, extract the year, and convert to integer type
df['Launched Year'] = pd.to_datetime(df['Launched Year'], format='%Y').dt.year.astype('int')

In [ ]:
df.info()

In [ ]:
# Mobile Weight column: Remove 'g' and convert to float
df['Mobile Weight (g)'] = df['Mobile Weight'].str.replace('g', '').astype(float)
df.drop('Mobile Weight', axis=1, inplace=True)

In [ ]:
# RAM column: Remove 'GB' and convert to float
def clean_ram(item):
  item = item.replace('GB', '')
  if '/' in item:
    item = item.split('/')[1] # Take the higher value

  return item

df['RAM (GB)'] = df['RAM'].apply(clean_ram).astype(float)
df.drop('RAM', axis=1, inplace=True)

In [ ]:
# Front Camera column: Remove 'MP' and convert to float
def clean_front_camera(item):
  item = [float(i) for i in re.findall(r'\d+', item)]
  return max(item) if item else 0.

df['Front Camera (MP)'] = df['Front Camera'].apply(clean_front_camera)
df.drop('Front Camera', axis=1, inplace=True)

In [ ]:
# Front Camera column: Remove 'MP', convert to float and split into 4 columns [Main Camera, Ultra Camera, Telephoto Camera, Macro Camera]
def clean_back_camera(item):
  items = item.split('+')
  list_camera = [0., 0., 0., 0.]

  for idx, item in enumerate(items):
    if idx==2:
      if 'macro' not in item.lower():
        list_camera[2] = float(item.split('MP')[0])
      elif 'macro' in item.lower():
        list_camera[3] = float(item.split('MP')[0])
    else:
      list_camera[idx] = float(item.split('MP')[0])

  return list_camera

df[['Main Camera (MP)', 'Ultra Camera (MP)', 'Telephoto Camera (MP)', 'Macro Camera (MP)']] = df['Back Camera'].apply(lambda x: pd.Series(clean_back_camera(x)))
df.drop('Back Camera', axis=1, inplace=True)

In [ ]:
# Battery Capacity column: Remove 'mAh' and convert to int
df['Battery Capacity (mAh)'] = df['Battery Capacity'].str.replace('mAh', '')
df['Battery Capacity (mAh)'] = df['Battery Capacity (mAh)'].str.replace(',', '').astype(int)
df.drop('Battery Capacity', axis=1, inplace=True)

In [ ]:
# Screen Size column: Remove 'inches' and convert to float
df['Screen Size (inches)'] = df['Screen Size'].apply(lambda x: x.split('inches')[0]).astype(float)
df.drop('Screen Size', axis=1, inplace=True)

In [ ]:
# Model Name column: Extract storage size column
def extract_storage(item):
  item = item.split(' ')[-1]
  if 'GB' in item:
    item = int(item.replace('GB', ''))
  elif 'TB' in item:
    item = int(item.replace('TB', '')) * 1024
  else:
    item = pd.NA

  return item

df['Storage (GB)'] = df['Model Name'].apply(lambda x: extract_storage(x))

In [ ]:
for col in df.columns:
  print(f'Unique values in {col}:\n', sorted(df[col].dropna().unique()), '\n')

In [ ]:
# Launched Price columns: we will create new fields for the country and currency
df['Launched Price (China)'] = df['Launched Price (China)'].str.replace('¥', 'CNY ').str.replace(',', '').str.replace('\xa0', '')

price_columns = [
    'Launched Price (Pakistan)', 'Launched Price (India)',
    'Launched Price (China)', 'Launched Price (USA)', 'Launched Price (Dubai)'
]

# Melt the DataFrame: Convert wide format into long format
df_melted = df.melt(id_vars=[col for col in df.columns if col not in price_columns],
                     value_vars=price_columns,
                     var_name="Country", value_name="Price")

# Extract Country from column names
df_melted["Country"] = df_melted["Country"].str.extract(r'Launched Price \((.*?)\)')
df_melted['Country'] = df_melted['Country'].astype("category")

# Now, df_melted has a single "Price" column and a "Country" column
print(df_melted[['Country', 'Price']].head())

# 🔹 Replace the original df with the transformed version
df = df_melted.copy()

In [ ]:
# Function to extract currency and numeric price
def split_currency_price(value):
    if pd.isna(value) or value.strip() == "":
        return pd.NA, pd.NA  # Handle missing values

    match = re.match(r'([A-Za-z]+)\s*([\d,]+)', str(value))  # Extract currency and price
    if match:
        currency = match.group(1)  # Extract currency (e.g., USD, PKR, INR)
        price = match.group(2).replace(',', '')  # Remove commas in numbers
        return currency, float(price)  # Convert price to float

    return pd.NA, pd.NA  # If no match, return NaN

# Apply function to split into two new columns
df[['Currency', 'Price (Numeric)']] = df['Price'].apply(lambda x: pd.Series(split_currency_price(x)))

# Now, df has separate 'Currency' and 'Price (Numeric)' columns
print(df[['Country', 'Price', 'Currency', 'Price (Numeric)']].head())

In [ ]:
# Convert 'Currency' to categorical
df['Currency'] = df['Currency'].astype('category')

# Convert 'Price (Numeric)' to integer
df['Price (Numeric)'] = df['Price (Numeric)'].astype('Int64')  # Supports NaN values

# Drop the original 'Price' column
df.drop(columns=['Price'], inplace=True)

In [ ]:
# Display rows where Currency is missing
missing_currency_rows = df[df['Currency'].isna()]
print(missing_currency_rows['Country'])

In [ ]:
df.loc[df['Country'] == 'Pakistan', 'Currency'] = 'PKR'
df.loc[df['Country'] == 'China', 'Currency'] = 'CNY'

In [ ]:
# Display rows where Currency is missing
missing_currency_rows = df[df['Currency'].isna()]
print(missing_currency_rows['Country'])

In [ ]:
# Define exchange rates to normalize prices using the currency column
exchange_rates = {
    'PKR': 0.0036,
    'INR': 0.012,
    'CNY': 0.14,
    'USD': 1.0,
    'AED': 0.27
}

# Function to normalize price
def normalize_price(row):
    if pd.isna(row['Price (Numeric)']) or pd.isna(row['Currency']):
        return np.nan  # Keep NaN if no price or currency
    return row['Price (Numeric)'] * exchange_rates.get(row['Currency'], np.nan)

# Apply normalization
df['Normalized Price (USD)'] = df.apply(normalize_price, axis=1)

# Now the prices are normalized
print(df[['Price (Numeric)', 'Currency', 'Normalized Price (USD)']].head())
# Drop the original 'Price Numeric' column
df.drop(columns=['Price (Numeric)'], inplace=True)

# Drop the currency column as it isn't needed any more
df.drop(columns=['Currency'], inplace=True)

In [ ]:
df.head(50)

In [ ]:
for col in df.columns:
  print(f'Unique values in {col}:\n', sorted(df[col].dropna().unique()), '\n')

In [ ]:
# Lets check and see after
df.info()

In [ ]:
df.isna().sum()

In [ ]:
# The normalized price seems to be missing two records, lets investigate and fillin using the average price grouped by country

# Ensure we are working on a copy to avoid SettingWithCopyWarning
df = df.copy()

# Calculate the average Normalized Price per Country
avg_price_per_country = df.groupby('Country', observed=True)['Normalized Price (USD)'].transform('mean')

# Fill missing values safely
df.loc[df['Normalized Price (USD)'].isna(), 'Normalized Price (USD)'] = avg_price_per_country

# Verify if there are still missing values
print(df['Normalized Price (USD)'].isna().sum())  # Should print 0 if all missing values are filled

In [ ]:
# Check summary statistics to look for inconsistancey and to verify
print(df[['Mobile Weight (g)', 'Screen Size (inches)', 'Battery Capacity (mAh)', 'Normalized Price (USD)']].describe())

In [ ]:
df[df['Mobile Weight (g)'] == 732]['Storage (GB)'].unique()

In [ ]:
# For Mobile Weight (g), Max = 732g seems very high (most phones weigh 150-250g)
# For Screen Size Max = 14.6 inches and a battery capacity of 11200 mAh is unusually large (could be tablets).
df[(df['Screen Size (inches)'] == 14.6) | (df['Mobile Weight (g)'] == 732)][['Storage (GB)', 'Screen Size (inches)', 'Battery Capacity (mAh)', 'Mobile Weight (g)']].drop_duplicates()

In [ ]:
# Lets investigate the max price of 39622, it is most likely to be an input error
df[df['Normalized Price (USD)'] == 39622][['Storage (GB)','Processor', 'Screen Size (inches)', 'Battery Capacity (mAh)', 'Normalized Price (USD)']].drop_duplicates()

In [ ]:
# Filter dataset for similar products (same processor, screen size close to 10.4 inches, and battery capacity close to 8200 mAh)
similar_products = df[
    (df['Processor'] == 'Unisoc T612') &
    (df['Screen Size (inches)'] == 10.4) &  # Screen size around 10.4 inches
    (df['Battery Capacity (mAh)'] == 8200)  # Battery capacity around 8200 mAh
]

# Display the filtered dataset with similar products
print(similar_products[['Storage (GB)', 'Processor', 'Screen Size (inches)', 'Battery Capacity (mAh)', 'Normalized Price (USD)']])

# Alternatively, you can check the price statistics of the similar products
print(similar_products['Normalized Price (USD)'].describe())

In [ ]:
# It seems like the $39622 is really an input error, lets deal with it

# Step 1: Remove the outlier
df_filtered = df[(df['Model Name'] != 'T21') | (df['Normalized Price (USD)'] != 39622)]

# Step 2: Group by relevant features and calculate the mean price
grouped = df_filtered.groupby(['Processor', 'Screen Size (inches)', 'Battery Capacity (mAh)'])['Normalized Price (USD)'].mean().reset_index()

# Step 3: Find the correct mean value for the T21 outlier
mean_price = grouped[(grouped['Processor'] == 'Unisoc T612') &
                     (grouped['Screen Size (inches)'] == 10.4) &
                     (grouped['Battery Capacity (mAh)'] == 8200)]['Normalized Price (USD)'].values[0]

# Step 4: Update the outlier's price
df.loc[(df['Model Name'] == 'T21') & (df['Normalized Price (USD)'] == 39622), 'Normalized Price (USD)'] = mean_price

# Step 5: Verify the update
updated_price = df[df['Model Name'] == 'T21'][['Model Name', 'Processor', 'Screen Size (inches)', 'Battery Capacity (mAh)', 'Normalized Price (USD)']]
print(updated_price)

In [ ]:
# Apply natural log transformation
df['log_normalized_price'] = np.log(df['Normalized Price (USD)'])

In [ ]:
df['log_normalized_price'].head()

#4: Vizualization data
1: Distribution of variables:

In [ ]:
# Histogram: Shows the frequency distribution of a continuous variable. This helps you see the shape of the distribution (e.g., normal, left-skewed, right-skewed, multi-peaked), the central values, and the spread of the data.
numerical_cols = ['Mobile Weight (g)',  'Screen Size (inches)',  'Battery Capacity (mAh)',
                  'Normalized Price (USD)',  'log_normalized_price',  'RAM (GB)',  'Front Camera (MP)',
                  'Main Camera (MP)',  'Ultra Camera (MP)',  'Telephoto Camera (MP)',
                  'Macro Camera (MP)',  'Storage (GB)',  'Launched Year']

for col in numerical_cols:
    plt.figure(figsize=(10, 6))
    sns.histplot(df[col], kde=True) # kde=True to display the Kernel density estimate line
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

Comment: Observe the distribution shape of 'Normalized Price (USD)' before and after applying log transformation (log_normalized_price). You will see that log transformation makes the distribution more symmetrical.

In [ ]:
# Box Plot: Displays the quartiles, medians, and potential outliers of  'Normalized Price (USD)',  'log_normalized_price'.

for col in [ 'Normalized Price (USD)',  'log_normalized_price']:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=df[col])
    plt.title(f'Box Plot  {col}')
    plt.xlabel(col)
    plt.show()

2:  Distribution of categorical variables:


In [ ]:
# Bar Chart: Shows the frequency or proportion of each category in a categorical variable.

categorical_cols = ['Company Name', 'Country']

for col in categorical_cols:
    plt.figure(figsize=(12, 6))
    sns.countplot(y=df[col], order=df[col].value_counts().index) # order to sort by frequency
    plt.title(f'Distribution of  {col}')
    plt.xlabel('Frequency')
    plt.ylabel(col)
    plt.show()

3. Relationship between variables:

In [ ]:
# Correlation Heatmap: Displays the correlation matrix between continuous variables.

for col in numerical_cols:
    if df[col].isnull().any():
        print(f"Column '{col}' has missing values. Filling with mean.")
        df[col] = df[col].fillna(df[col].mean())

correlation_matrix = df[numerical_cols].corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('The correlation matrix')
plt.show()

Both Normalized Price (USD) and log_normalized_price show significant positive correlations with factors such as RAM, storage, year of launch, battery capacity, and screen size. This makes economic and technical sense, as higher-end phones with better features typically cost more.

In [ ]:
# Box Plot by Group: Displays the distribution of Normalized Price (USD) across categories of a categorical variable.

plt.figure(figsize=(15, 8))
sns.boxplot(x='Company Name', y='Normalized Price (USD)', data=df)
plt.title('Price by phone brand')
plt.xlabel('Phone brand')
plt.ylabel('Price (USD)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Box Plot by Group: Displays the distribution of log_normalized_price across categories of a categorical variable.

plt.figure(figsize=(15, 8))
sns.boxplot(x='Company Name', y='log_normalized_price', data=df)
plt.title('Price by phone brand')
plt.xlabel('Phone brand')
plt.ylabel('Price')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

4. Trends over time

In [ ]:
yearly_avg_price = df.groupby('Launched Year')['Normalized Price (USD)'].mean()
plt.figure(figsize=(10, 6))
plt.plot(yearly_avg_price.index, yearly_avg_price.values, marker='o')
plt.title('Average price by launched year')
plt.xlabel('Launched Year')
plt.ylabel('Average price (USD)')
plt.grid(True)
plt.show()

In [ ]:
# For each Company
plt.figure(figsize=(12, 8))

for company in df['Company Name'].unique():
    company_data = df[df['Company Name'] == company]
    yearly_avg_price_company = company_data.groupby('Launched Year')['Normalized Price (USD)'].mean()
    plt.plot(yearly_avg_price_company.index, yearly_avg_price_company.values, marker='o', label=company)

plt.title('Average price by launched year and Company Name')
plt.xlabel('Launched Year')
plt.ylabel('Average price (USD)')
plt.legend(title='Company Name', bbox_to_anchor=(1.05, 1), loc='upper left') # Show legend outside the chart
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Using FacetGrid for compare across countries

g = sns.FacetGrid(df, col='Country', col_wrap=3, height=4, aspect=1.5)
g.map(sns.lineplot, 'Launched Year', 'Normalized Price (USD)', 'Company Name', marker='o', errorbar=None)
g.set_axis_labels('Launched Year', 'Average Price (USD)')
g.set_titles(col_template="{col_name}")
g.add_legend(title='Company Name', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout(rect=[0, 0, 0.9, 1]) # Adjust layout to make room for legend
plt.show()

In [ ]:
# Prepare data (convert categorical to numeric if not already)
df_encoded = df.copy()
for col in ['Company Name', 'Model Name', 'Processor', 'Country']:
    if df_encoded[col].dtype == 'category':
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col])

X = df_encoded.drop(['Normalized Price (USD)', 'log_normalized_price'], axis=1)
y = df_encoded['Normalized Price (USD)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

feature_importances = model.feature_importances_
feature_names = X.columns

importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importances_df = importances_df.sort_values(by='Importance', ascending=False)
print(importances_df)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importances_df)
plt.title('Feature Importance from Random Forest')
plt.show()

In [ ]:
df['Processor'].value_counts()

# Prepare data for the model:

In [ ]:
df['Processor'].value_counts()

1: Based on the value_counts() result of the 'Processor' column, you see that there are many different processors, and most of them appear with very low frequency (only 5 times). This confirms the concern that one-hot encoding this column has created a large number of uninformative features, which can cause dimensionality and overfitting issues.

2: The idea: Instead of coding each processor model individually, you can group them by major manufacturer (e.g. Qualcomm Snapdragon, MediaTek Helio/Dimensity, Apple Bionic, Samsung Exynos, Google Tensor, Unisoc, Kirin).

In [ ]:
# Grouping of processors based on brand:

def extract_processor_brand(processor):
    if 'Snapdragon' in processor:
        return 'Qualcomm Snapdragon'
    elif 'MediaTek' in processor or 'Dimensity' in processor or 'Helio' in processor:
        return 'MediaTek'
    elif 'Bionic' in processor or 'A' in processor:
        return 'Apple Bionic'
    elif 'Exynos' in processor:
        return 'Samsung Exynos'
    elif 'Tensor' in processor:
        return 'Google Tensor'
    elif 'Unisoc' in processor:
        return 'Unisoc'
    elif 'Kirin' in processor:
        return 'Kirin'
    else:
        return 'Other' # For rare cases

df['Processor_Brand'] = df['Processor'].apply(extract_processor_brand)
df = pd.get_dummies(df, columns=['Processor_Brand'], drop_first=True)
df.drop('Processor', axis=1, inplace=True)

In [ ]:
# Encoding categorical variables:

df = pd.get_dummies(df, columns=['Company Name', 'Country'], drop_first=False)
for col in df.columns:
  print(col)

# 6: Split Data: Split the selected dataset into training set and test set.

In [ ]:
numerical_features = [
    'Launched Year', 'Mobile Weight (g)', 'RAM (GB)', 'Front Camera (MP)',
    'Main Camera (MP)', 'Ultra Camera (MP)', 'Telephoto Camera (MP)',
    'Macro Camera (MP)', 'Battery Capacity (mAh)', 'Screen Size (inches)',
    'Storage (GB)'
]
X = df[numerical_features].copy()
y = df['log_normalized_price'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# joblib.dump(X_train_scaled.columns, os.path.join(models_dir, 'feature_columns.joblib'))

#7: Training model

In [ ]:
np.random.seed(42)
num_samples = 100

pipelines = {
    'LinearRegression': Pipeline([
        ('regressor', LinearRegression())
    ]),
    'RandomForestRegressor': Pipeline([
        ('regressor', RandomForestRegressor(random_state=42))
    ]),
    'XGBRegressor': Pipeline([
        ('regressor', XGBRegressor(random_state=42, eval_metric='rmse', use_label_encoder=False))
    ]),
    'LGBMRegressor': Pipeline([
        ('regressor', LGBMRegressor(random_state=42))
    ])
}


In [ ]:
param_grids = {
    'LinearRegression': {
        'regressor__fit_intercept': [True, False]
    },
    'RandomForestRegressor': {
        'regressor__n_estimators': [100, 200],
        'regressor__max_depth': [None, 10, 20],
        'regressor__min_samples_split': [2, 5]
    },
    'XGBRegressor': {
        'regressor__n_estimators': [100, 200],
        'regressor__learning_rate': [0.05, 0.1],
        'regressor__max_depth': [3, 5]
    },
    'LGBMRegressor': {
        'regressor__n_estimators': [100, 200],
        'regressor__learning_rate': [0.05, 0.1],
        'regressor__num_leaves': [31, 62]
    }
}


In [ ]:
best_models = {}
results = []

for model_name, pipeline in pipelines.items():
    print(f"\nĐang huấn luyện {model_name}...")
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grids[model_name],
        cv=5, # Số fold cho cross-validation
        scoring='neg_mean_squared_error', # Sử dụng RMSE (âm) để tìm kiếm tốt nhất
        n_jobs=-1, # Sử dụng tất cả các core CPU
        verbose=1 # Hiển thị tiến trình
    )

    # Huấn luyện trên dữ liệu đã được scale
    grid_search.fit(X_train_scaled, y_train)

    best_estimator = grid_search.best_estimator_
    best_params = grid_search.best_params_
    best_score = -grid_search.best_score_ # Chuyển đổi lại về RMSE dương

    print(f"Mô hình tốt nhất cho {model_name}:")
    print(f"  Tham số tốt nhất: {best_params}")
    print(f"  RMSE trên tập huấn luyện (CV): {np.sqrt(best_score):.4f}")

    # Đánh giá trên tập kiểm tra (đã được scale)
    y_pred = best_estimator.predict(X_test_scaled)
    mse_test = mean_squared_error(y_test, y_pred)
    rmse_test = np.sqrt(mse_test)
    r2_test = r2_score(y_test, y_pred)

    print(f"  RMSE trên tập kiểm tra: {rmse_test:.4f}")
    print(f"  R-squared trên tập kiểm tra: {r2_test:.4f}")

    best_models[model_name] = best_estimator
    results.append({
        'Model': model_name,
        'Best Parameters': best_params,
        'Train RMSE (CV)': np.sqrt(best_score),
        'Test RMSE': rmse_test,
        'Test R-squared': r2_test
    })

# --- 4. Tóm tắt kết quả ---

print("\n--- Tóm tắt kết quả ---")
results_df = pd.DataFrame(results)
print(results_df.sort_values(by='Test RMSE'))

# Bạn có thể truy cập các mô hình tốt nhất như sau:
# best_linear_regression = best_models['LinearRegression']
# best_random_forest = best_models['RandomForestRegressor']
# best_xgb = best_models['XGBRegressor']
# best_lgbm = best_models['LGBMRegressor']

#Conclusion:

- LinearRegression is not suitable for this problem, it only explains less than 50% of the variation in log-normalized prices.

- Ensemble tree-based models (RandomForest, XGBoost, LightGBM) give superior performance, explaining more than 90% of the variation. This confirms that the relationship between features and phone prices is non-linear and complex, which tree models can capture better.

- LGBMRegressor is the best model among the tested models, achieving the lowest RMSE and highest R-squared on the test set.

In [ ]:
# Đường dẫn đến thư mục models
models_dir = 'models'
os.makedirs(models_dir, exist_ok=True) # Tạo thư mục nếu chưa tồn tại

# Lưu mô hình LGBMRegressor tốt nhất
best_lgbm_model = best_models['LGBMRegressor']
model_path = os.path.join(models_dir, 'best_lgbm_regressor.joblib')
joblib.dump(best_lgbm_model, model_path)
print(f"Mô hình LGBMRegressor đã được lưu tại: {model_path}")

# Lưu StandardScaler
scaler_path = os.path.join(models_dir, 'scaler.joblib')
joblib.dump(scaler, scaler_path) # 'scaler' là biến StandardScaler bạn đã tạo trước đó
print(f"StandardScaler đã được lưu tại: {scaler_path}")

joblib.dump(X_train_scaled.columns, os.path.join(models_dir, 'feature_columns.joblib'))

In [ ]:
def predict_new_phone_price(new_phone_data_raw, models_dir='models'):
    """
    Dự đoán giá điện thoại mới dựa trên dữ liệu đầu vào thô.

    Args:
        new_phone_data_raw (dict): Một dictionary chứa dữ liệu về điện thoại mới.
                                   Các khóa phải khớp với tên cột gốc trước khi tiền xử lý.
                                   Ví dụ: {'Mobile Weight': '180g', 'RAM': '8GB', ...}
        models_dir (str): Thư mục chứa mô hình và scaler đã lưu.

    Returns:
        float: Giá điện thoại dự đoán (trên thang giá gốc, không phải logarit).
        None: Nếu có lỗi trong quá trình xử lý hoặc dự đoán.
    """
    # Load mô hình và scaler đã lưu
    try:
        best_lgbm_model = joblib.load(os.path.join(models_dir, 'best_lgbm_regressor.joblib'))
        scaler = joblib.load(os.path.join(models_dir, 'scaler.joblib'))
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy mô hình hoặc scaler trong thư mục '{models_dir}'.")
        return None

    # Tạo DataFrame từ dữ liệu thô
    df_new = pd.DataFrame([new_phone_data_raw])

    # --- Các bước tiền xử lý giống hệt như trong quá trình huấn luyện ---
    # Bạn phải đảm bảo rằng các bước này khớp chính xác với code tiền xử lý của bạn

    # 1. Chuyển đổi 'Launched Year'
    df_new['Launched Year'] = pd.to_datetime(df_new['Launched Year'], format='%Y').dt.year.astype('int')

    # 2. Xử lý 'Mobile Weight (g)'
    df_new['Mobile Weight (g)'] = df_new['Mobile Weight'].str.replace('g', '').astype(float)
    df_new.drop('Mobile Weight', axis=1, inplace=True) # Xóa cột gốc

    # 3. Xử lý 'RAM (GB)'
    def clean_ram(item):
        item = item.replace('GB', '')
        if '/' in item:
            item = item.split('/')[1]
        return float(item)
    df_new['RAM (GB)'] = df_new['RAM'].apply(clean_ram)
    df_new.drop('RAM', axis=1, inplace=True)

    # 4. Xử lý 'Front Camera (MP)'
    def clean_front_camera(item):
        item = [float(i) for i in re.findall(r'\d+', str(item))]
        return max(item) if item else 0.
    df_new['Front Camera (MP)'] = df_new['Front Camera'].apply(clean_front_camera)
    df_new.drop('Front Camera', axis=1, inplace=True)

    # 5. Xử lý 'Back Camera' thành 4 cột
    def clean_back_camera(item):
        items = item.split('+')
        list_camera = [0., 0., 0., 0.]
        for idx, sub_item in enumerate(items):
            if idx == 2:
                if 'macro' not in sub_item.lower():
                    list_camera[2] = float(sub_item.split('MP')[0])
                elif 'macro' in sub_item.lower():
                    list_camera[3] = float(sub_item.split('MP')[0])
            else:
                list_camera[idx] = float(sub_item.split('MP')[0])
        return list_camera

    # Tạo các cột tạm thời trước khi gán
    temp_cols = ['Main Camera (MP)', 'Ultra Camera (MP)', 'Telephoto Camera (MP)', 'Macro Camera (MP)']
    df_new[temp_cols] = df_new['Back Camera'].apply(lambda x: pd.Series(clean_back_camera(x)))
    df_new.drop('Back Camera', axis=1, inplace=True)

    # 6. Xử lý 'Battery Capacity (mAh)'
    df_new['Battery Capacity (mAh)'] = df_new['Battery Capacity'].str.replace('mAh', '').str.replace(',', '').astype(int)
    df_new.drop('Battery Capacity', axis=1, inplace=True)

    # 7. Xử lý 'Screen Size (inches)'
    df_new['Screen Size (inches)'] = df_new['Screen Size'].apply(lambda x: x.split('inches')[0]).astype(float)
    df_new.drop('Screen Size', axis=1, inplace=True)

    # 8. Trích xuất 'Storage (GB)' từ 'Model Name'
    def extract_storage(item):
        item = str(item).split(' ')[-1] # Đảm bảo là string
        if 'GB' in item:
            return int(item.replace('GB', ''))
        elif 'TB' in item:
            return int(item.replace('TB', '')) * 1024
        else:
            return pd.NA

    df_new['Storage (GB)'] = df_new['Storage'].apply(extract_storage)

    # 9. Xử lý Processor_Brand (đây là một bước quan trọng vì bạn đã thay thế Processor gốc)
    def extract_processor_brand(processor):
        if 'Snapdragon' in processor: return 'Qualcomm Snapdragon'
        elif 'MediaTek' in processor or 'Dimensity' in processor or 'Helio' in processor: return 'MediaTek'
        elif 'Bionic' in processor or 'A' in processor: return 'Apple Bionic'
        elif 'Exynos' in processor: return 'Samsung Exynos'
        elif 'Tensor' in processor: return 'Google Tensor'
        elif 'Unisoc' in processor: return 'Unisoc'
        elif 'Kirin' in processor: return 'Kirin'
        else: return 'Other'
    df_new['Processor_Brand'] = df_new['Processor'].apply(extract_processor_brand)
    df_new.drop('Processor', axis=1, inplace=True) # Xóa cột gốc

    # 10. One-Hot Encoding cho Processor_Brand, Company Name, Country
    # Cần tạo các cột OHE giống hệt như tập huấn luyện.
    # Cách tốt nhất là tạo một DataFrame với tất cả các category có thể có
    # và sau đó join hoặc reindex df_new.
    # Trong ví dụ này, tôi sẽ giả định bạn đã có danh sách tất cả các cột OHE từ X_train.
    # Nếu không, bạn cần lưu trữ danh sách các cột này trong quá trình huấn luyện.

    # Lấy danh sách đầy đủ các cột (features) mà mô hình được huấn luyện trên đó.
    # Điều này cực kỳ quan trọng để đảm bảo thứ tự và sự đầy đủ của các cột.
    # Trong code huấn luyện của bạn, `X_train_scaled.columns` chứa tất cả các cột cần thiết.
    # Bạn cần lưu trữ danh sách này.

    # Giả định: Bạn đã lưu danh sách các cột như sau trong quá trình huấn luyện:
    # joblib.dump(X_train_scaled.columns, os.path.join(models_dir, 'feature_columns.joblib'))
    try:
        trained_feature_columns = joblib.load(os.path.join(models_dir, 'feature_columns.joblib'))
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy danh sách cột đặc trưng đã huấn luyện. Vui lòng lưu `X_train_scaled.columns`.")
        return None

    # Tách các cột số và cột OHE
    numerical_features_list = [
        'Launched Year', 'Mobile Weight (g)', 'RAM (GB)', 'Front Camera (MP)',
        'Main Camera (MP)', 'Ultra Camera (MP)', 'Telephoto Camera (MP)',
        'Macro Camera (MP)', 'Battery Capacity (mAh)', 'Screen Size (inches)',
        'Storage (GB)'
    ]

    # Tạo các cột OHE cho df_new
    # Danh sách các cột phân loại (sau khi bạn đã tạo Processor_Brand)
    categorical_cols_for_ohe = ['Processor_Brand', 'Company Name', 'Country']

    # Tạo DataFrame tạm thời để chứa OHE
    df_ohe_temp = pd.get_dummies(df_new[categorical_cols_for_ohe], drop_first=False)

    # Kết hợp các cột số và các cột OHE
    # Đảm bảo df_new chỉ chứa các cột cần thiết trước khi concat
    df_processed = pd.concat([df_new[numerical_features_list], df_ohe_temp], axis=1)

    # Đảm bảo df_processed có chính xác các cột như `trained_feature_columns`
    # Đây là bước cực kỳ quan trọng để xử lý các category chưa thấy.
    # Tạo một DataFrame trống với các cột mong muốn, sau đó gán dữ liệu.
    X_processed = pd.DataFrame(columns=trained_feature_columns)

    for col in trained_feature_columns:
        if col in df_processed.columns:
            X_processed[col] = df_processed[col]
        else:
            X_processed[col] = 0 # Điền 0 cho các category chưa thấy

    # Xử lý NaN (có thể xảy ra nếu Storage (GB) không trích xuất được)
    # Bạn đã dùng df[col] = df[col].fillna(df[col].mean()) trong phần visualization
    # Nhưng trong prediction, không có mean của toàn bộ tập dữ liệu.
    # Tốt nhất là xử lý NaN ở đây. Ví dụ, điền 0 hoặc một giá trị mặc định hợp lý.
    # Hoặc đảm bảo rằng dữ liệu đầu vào không có NaN cho các cột số.
    # Ví dụ:
    for col in numerical_features_list:
        if col in X_processed.columns and X_processed[col].isna().any():
            print(f"Cảnh báo: Cột '{col}' có giá trị thiếu. Đang điền 0.")
            X_processed[col] = X_processed[col].fillna(0) # Hoặc giá trị mặc định khác

    # Sắp xếp lại cột theo thứ tự của X_train_scaled.columns
    X_processed = X_processed[trained_feature_columns]

    # Chuẩn hóa dữ liệu bằng scaler đã lưu
    X_processed_scaled = scaler.transform(X_processed)

    # Dự đoán (trên thang logarit)
    log_predicted_price = best_lgbm_model.predict(X_processed_scaled)[0]

    # Chuyển đổi ngược về thang giá gốc
    predicted_price = np.exp(log_predicted_price)

    return predicted_price

# --- Ví dụ cách sử dụng hàm dự đoán ---
# Dữ liệu mẫu (thay thế bằng dữ liệu điện thoại thực tế của bạn)
# RẤT QUAN TRỌNG: Các khóa trong dictionary này phải là TÊN CỘT GỐC của bạn
# trước khi các bước làm sạch và tiền xử lý bắt đầu.
new_phone_data = {
    'Launched Year': 2024,
    'Mobile Weight': '190g',
    'RAM': '12GB',
    'Front Camera': '32MP',
    'Back Camera': '108MP+12MP+5MP', # Ví dụ: Main+Ultra+Macro (không Telephoto)
    'Battery Capacity': '5000mAh',
    'Screen Size': '6.7 inches',
    'Storage': '256GB',
    'Processor': 'Qualcomm Snapdragon 8 Gen 3',
    'Company Name': 'Samsung',
    'Country': 'USA' # Phải là một trong các quốc gia bạn đã huấn luyện
}

# Trước khi chạy hàm này, hãy đảm bảo bạn đã lưu `X_train_scaled.columns`
# trong quá trình huấn luyện của bạn để hàm có thể tải nó lên.
# Ví dụ, sau dòng `X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)`
# Bạn thêm dòng sau:
# joblib.dump(X_train_scaled.columns, os.path.join(models_dir, 'feature_columns.joblib'))


# Sau khi bạn đã chạy code trên để lưu mô hình và scaler:
predicted_price = predict_new_phone_price(new_phone_data)

if predicted_price is not None:
    print(f"\nGiá dự đoán của điện thoại mới là: ${predicted_price:.2f}")